In [1]:
import os,sys, math, ROOT
from array import array
import numpy as np
# %jsroot on

#Importing a helper file where some functions are defined
from helper import *

#Input file
inputfile = ROOT.TFile("../Data/data.root","READ")

#Output file
outputfile = ROOT.TFile("output.root","RECREATE")

#Accessing the "TTree" from root that contains the data (one entry per event)
tree = inputfile.Get("ntuplizer/tree")

nentries = tree.GetEntries()
print("Number of entries: ", nentries)

Welcome to JupyROOT 6.26/06
Number of entries:  104317


The goal of this file is to create and then compare histograms of the invariant masses involved in the
\begin{equation}
    H^0\to l^+l^- + l'^+l'^-
\end{equation}
process, where $l$ is taken to be either an electron or a muon.

To better understand the result, we will also consider separately the following process
\begin{equation}
    H^0\to l^+l^-
\end{equation}

The code snippet underneath defines several `histogram` objects. The name given to those objects follows the following logic:
- `hist_mass` for the notion that the histogram will display the invariant mass of system of particles;
- `_X` shows the type of particles detected from which we compute an invariant mass
- The extra mention of `_zoom` or `_ZoomExtra` concerns the range on which the invariant mass is computed. `_zoom` means one computes and displays the invariant mass in the interval $[80,100] GeV$ whilst `_ZoomExtra` means the displayed interval is $[0,10] GeV$.
- Additionnaly, the mention of `_Higgs` means one is trying to truncate the data to display only the process of interest to us: the apparition of the Higgs boson!

In [2]:
#Example of histograms 
#The arguments are the following: 
#"name", "title;x axis title;yaxis title", nb of bins, min value, max value

hist_mass_2e2mu = ROOT.TH1F("hist_mass_2e2mu",";Four Lepton Invariant Mass [GeV];Events", 500, 0, 500)
hist_mass_2e = ROOT.TH1F("hist_mass_2e",";Electron pair Invariant Mass [GeV];Events", 500, 0, 500)
hist_mass_2mu = ROOT.TH1F("hist_mass_2mu",";Electron pair Invariant Mass [GeV];Events", 500, 0, 500)


# Zoom in the low mass regions
hist_mass_2e2mu_zoom = ROOT.TH1F("hist_mass_2e2mu_zoom",";Four Lepton Invariant Mass [GeV];Events", 500, 80, 100)
hist_mass_2e_zoom = ROOT.TH1F("hist_mass_2e_zoom",";Electron pair Invariant Mass [GeV];Events", 500, 80, 100)
hist_mass_2mu_zoom = ROOT.TH1F("hist_mass_2mu_zoom",";Muon pair Invariant Mass [GeV];Events", 500, 80, 100)

hist_mass_2e2mu_ZoomExtra = ROOT.TH1F("hist_mass_2e2mu_ZoomExtra",";Four Lepton Invariant Mass [GeV];Events", 500, 0, 10)
hist_mass_2e_ZoomExtra = ROOT.TH1F("hist_mass_2e_ZoomExtra",";Electron pair Invariant Mass [GeV];Events", 500, 0, 10)
hist_mass_2mu_ZoomExtra = ROOT.TH1F("hist_mass_2mu_ZoomExtra",";Muon pair Invariant Mass [GeV];Events", 500, 0, 10)

# Zoom in the Higgs region
# hist_mass_2e2mu_Higgs = ROOT.TH1F("hist_mass_2e2mu_Higgs",";Four Lepton Invariant Mass [GeV];Events", 500, 100, 150)
# hist_mass_2e_Higgs = ROOT.TH1F('hist_mass_2e_Higgs',";Electron pair Invariant Mass [GeV];Events",500,100,150)
# hist_mass_2mu_Higgs = ROOT.TH1F('hist_mass_2mue_Higgs',";Muon pair Invariant Mass [GeV];Events",500,100,150)

In [3]:
#Loop over entries
invariant_2e2u = np.array([])
for i in range(0,nentries):
    tree.GetEntry(i) # Load entry number i
    n_e,n_mu_minus,n_mu_p,n_p = 0,0,0,0   

    for lep in range(0,len(tree._lPt)): # Loop over leptons per event
        if tree._lpdgId[lep] == 11 and n_e != 1: # Detect if lepton == electron
            n_e += 1
            Index_e = lep
        elif tree._lpdgId[lep] == -11 and n_p != 1: # Detect if lepton == positron
            n_p += 1
            Index_p = lep
        elif tree._lpdgId[lep] == 13 and n_mu_minus != 1: # Detect if lepton == muon-
            n_mu_minus += 1
            Index_mu_minus = lep
        elif tree._lpdgId[lep] == -13 and n_mu_p != 1: # Detect if lepton == muon+
            n_mu_p +=1
            Index_mu_plus = lep
        if (n_e,n_p,n_mu_minus,n_mu_p) == (1,1,1,1): # If all sought-after leptons are detected, break out of the loop
            invariant_mass = invariantmass4l(tree,Index_e,Index_p,Index_mu_minus,Index_mu_plus)
            invariant_2e2u = np.append(invariant_2e2u,invariant_mass)
            #print(invariant_mass)
            hist_mass_2e2mu.Fill(invariant_mass)
            # hist_mass_2e2mu_Higgs.Fill(invariant_mass)

In [4]:
#Draw the histogram and the canvas
c = ROOT.TCanvas("canvas","c_NPV",600,600)
# hist_n_mu.SetLineColor(ROOT.kRed)
# hist_n_e.Draw()
# hist_n_mu.Draw("same")
# hist_npv.Draw()
# c.Draw()
# savehisto(outputfile, hist_n_mu, "hist_n_mu_log")
# savehisto(outputfile,hist_n_e,"hist_n_e_log")

savehisto(outputfile,hist_mass_2e2mu,"hist_2e2mu_InvariantMass",logy=True)
# savehisto(outputfile,hist_mass_2e2mu_Higgs,"hist_2e2mu_Higgs_InvariantMass",logy=True)

Info in <TCanvas::Print>: pdf file pdfs/hist_2e2mu_InvariantMass.pdf has been created


There are two interesting peaks observed here. One slightly under $100 GeV$ whilst the second is around $125GeV$. 

The latter corresponds to the Higgs boson. This is in alignment with the scientific litterature on the matter:
- $125.38 \pm 0.14 GeV$ (CMS 2018)
- $125.35 \pm 0.15 GeV$ (CMS 2019)
- $124.97 \pm 0.24 GeV$ (ATLAS 2018)

Let us take a look into the first peak and the processes behind it.

# Invariant mass for a pair

Let us compare how the invariant mass of each pair differ. Based on the result, we zoom in different regions to see how they compare and if the same processes arise.

## 1. Pair of electrons

We consider the following relation
\begin{equation}
    H^0\to e^+e^-
\end{equation}


In [5]:
#Loop over entries
invariant_2e = np.array([])
for i in range(0,nentries):
    tree.GetEntry(i) # Load entry number i
    n_e,n_mu_minus,n_mu_p,n_p = 0,0,0,0   

    for lep in range(0,len(tree._lPt)): # Loop over leptons per event
        if tree._lpdgId[lep] == 11 and n_e != 1: # Detect if lepton == electron
            n_e += 1
            Index_e = lep
        elif tree._lpdgId[lep] == -11 and n_p != 1: # Detect if lepton == positron
            n_p += 1
            Index_p = lep
        elif tree._lpdgId[lep] == 13 and n_mu_minus != 1: # Detect if lepton == muon-
            n_mu_minus += 1
            Index_mu_minus = lep
        elif tree._lpdgId[lep] == -13 and n_mu_p != 1: # Detect if lepton == muon+
            n_mu_p +=1
            Index_mu_plus = lep
        if (n_e,n_p)== (1,1): # If all sought-after leptons are detected, break out of the loop
            invariant_mass = invariantmass2l(tree,Index_e,Index_p)
            invariant_2e = np.append(invariant_2e,invariant_mass)
            # print(invariant_mass)
            hist_mass_2e.Fill(invariant_mass)
            hist_mass_2e_zoom.Fill(invariant_mass)
            hist_mass_2e_ZoomExtra.Fill(invariant_mass)
            # hist_mass_2e_Higgs.Fill(invariant_mass)

In [6]:
savehisto(outputfile,hist_mass_2e,"hist_2e_InvariantMass",logy=True)
savehisto(outputfile,hist_mass_2e_zoom,"hist_2e_InvariantMass_zoom",logy=True)
savehisto(outputfile,hist_mass_2e_ZoomExtra,"hist_2e_InvariantMass_ZoomExtra",logy=True)
# savehisto(outputfile,hist_mass_2e_Higgs,"hist_2e_Higgs_InvariantMass",logy=True)

Info in <TCanvas::Print>: pdf file pdfs/hist_2e_InvariantMass.pdf has been created
Info in <TCanvas::Print>: pdf file pdfs/hist_2e_InvariantMass_zoom.pdf has been created
Info in <TCanvas::Print>: pdf file pdfs/hist_2e_InvariantMass_ZoomExtra.pdf has been created


## 2. Pair of muons

We consider the following relation
\begin{equation}
    H^0\to \mu^+\mu^-
\end{equation}

In [7]:
#Loop over entries
invariant_2m = np.array([])
for i in range(0,nentries):
    tree.GetEntry(i) # Load entry number i
    n_e,n_mu_minus,n_mu_p,n_p = 0,0,0,0   

    for lep in range(0,len(tree._lPt)): # Loop over leptons per event
        if tree._lpdgId[lep] == 11 and n_e != 1: # Detect if lepton == electron
            n_e += 1
            Index_e = lep
        elif tree._lpdgId[lep] == -11 and n_p != 1: # Detect if lepton == positron
            n_p += 1
            Index_p = lep
        elif tree._lpdgId[lep] == 13 and n_mu_minus != 1: # Detect if lepton == muon-
            n_mu_minus += 1
            Index_mu_minus = lep
        elif tree._lpdgId[lep] == -13 and n_mu_p != 1: # Detect if lepton == muon+
            n_mu_p +=1
            Index_mu_plus = lep
        if (n_mu_minus,n_mu_p)== (1,1): # If all sought-after leptons are detected, break out of the loop
            invariant_mass = invariantmass2l(tree,Index_mu_minus,Index_mu_plus)
            invariant_2m = np.append(invariant_2m,invariant_mass)
            # print(invariant_mass)
            hist_mass_2mu.Fill(invariant_mass)
            hist_mass_2mu_zoom.Fill(invariant_mass)
            hist_mass_2mu_ZoomExtra.Fill(invariant_mass)
            # hist_mass_2mu_Higgs.Fill(invariant_mass)

In [8]:
savehisto(outputfile,hist_mass_2mu,"hist_2mu_InvariantMass",logy=True)
savehisto(outputfile,hist_mass_2mu_zoom,"hist_2mu_InvariantMass_zoom",logy=True)
savehisto(outputfile,hist_mass_2mu_ZoomExtra,"hist_2mu_InvariantMass_ZoomExtra",logy=True)
# savehisto(outputfile,hist_mass_2mu_Higgs,"hist_2mu_Higgs_InvariantMass_ZoomExtra",logy=True)

Info in <TCanvas::Print>: pdf file pdfs/hist_2mu_InvariantMass.pdf has been created
Info in <TCanvas::Print>: pdf file pdfs/hist_2mu_InvariantMass_zoom.pdf has been created
Info in <TCanvas::Print>: pdf file pdfs/hist_2mu_InvariantMass_ZoomExtra.pdf has been created


# 3. Comparison of the results

Having created and compared different histograms in different regions, let us note the following remarks:

- In the $[0,10] Gev$ region, there are no clear peak observed for the $e^+e^-$ pair. This however does not hold true in the muon pair, as a peak appears around $3.25\pm 2.47 GeV$.
- In the $[80,100] Gev$ region, a peak appears for both peaks. The peaks should correspond to the same process, as their invariant mass seems to the same: $90.47\pm 3.59 Gev$ for the $\mu^+\mu^-$ pair and $89.78\pm 3.72 Gev$ for the electron pair.